In [1]:
local_run = False
store_local = False
local_path = "/Users/algin/Greenwich/MSc Project/project_code/hate_speech_detection/data/mixed/"

In [2]:
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

Defaulting to user installation because normal site-packages is not writeable
  Using cached emoji-2.0.0-py3-none-any.whl


In [4]:
import pandas as pd
import numpy as np
import random
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

if local_run:
    nltk.data.path.append('/Users/algin/VOLD/nltk_data')
    nltk.download('stopwords',download_dir='/Users/algin/VOLD/nltk_data')
    nltk.download('punkt',download_dir='/Users/algin/VOLD/nltk_data')
else:
    nltk.download('stopwords')
    nltk.download('punkt')
stop_words = set(stopwords.words('english'))


ModuleNotFoundError: ignored

In [ ]:
train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/3-model-experiments/data/cleaned/cleaned_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/3-model-experiments/data/cleaned/cleaned_test_data.csv?raw=true")

In [ ]:
train_data.head()

,tweet_text,cyberbullying_type,is_cyberbullying,cleaned
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0,word katandandr food crapilici
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0,aussietv white
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0,classi whore red velvet cupcak
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0,meh p thank head up concern anoth angri dude t...
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0,isi account pretend kurdish account like islam...


In [ ]:
test_data.head()

,tweet_text,actual_value,is_cyberbullying,cleaned
0,"""\n\nI decided on """"breached"""" as a term to de...",[],0,decid breach term describ damag dam
1,Support Adding information regarding the 2007 ...,[],0,support ad inform regard 2007 easter denial se...
2,Stick to topic \n\nI would suggest cleaning up...,[],0,stick topic would suggest clean articl contain...
3,20 moments where e.g. or i.e. is used in an ar...,[],0,20 moment eg ie use articl print 40 page exces...
4,"""\n\n ADHD \n\nIt was a fight there for a whil...",[],0,adhd fight while thank scuro left seen month l...


In [ ]:
train_data['length'] = train_data['tweet_text'].str.len()
test_data['length'] = test_data['tweet_text'].str.len()

train_data['word_count'] = train_data['tweet_text'].str.split().str.len()
test_data['word_count'] = test_data['tweet_text'].str.split().str.len()

In [ ]:
train_data['word_count'].mean(), test_data['word_count'].mean()

(26.07198050165388, 65.58979124964254)

In [ ]:
train_data['length'].mean(), test_data['length'].mean()

(149.36787058240597, 383.1627108950529)

In [ ]:
train_data['length'].max(), test_data['length'].max()

(4962, 5000)

In [ ]:
train_data['word_count'].max(), test_data['word_count'].max()

(790, 1403)

In [ ]:
pred_issue_values = [['malignant'],['rude'],['abuse'],['malignant', 'rude', 'abuse'],['malignant', 'rude'],['malignant', 'abuse'],['rude', 'abuse'],
                     ['malignant', 'highly_malignant', 'rude', 'abuse'],['malignant', 'rude', 'abuse', 'loathe'],['malignant', 'threat'],
                     ['malignant', 'loathe'],['malignant', 'abuse', 'loathe'] ]

**Create mixed datatset**

In [ ]:
malignant_raw_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/raw/malignant_comments/train.csv?raw=true")
unbalanced_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/unbalanced/unbalanced_train_data.csv?raw=true")

In [ ]:
len(malignant_raw_data)

159571

In [ ]:
len(unbalanced_train_data)

47692

In [ ]:
# Function to convert malignant data into a format to create new column 'is_cyberbullying'
def convert_malignant_tweets_format(data):
    new_data = pd.DataFrame()
    new_data['tweet_text'] = data['comment_text']
    new_data['actual_value'] = data.apply(lambda row: row[row == 1].index.tolist(), axis=1)
    new_data['is_cyberbullying'] = np.where(new_data['actual_value'].apply(lambda x:len(x)) == 0,0,1)
    return new_data

In [ ]:
malignant_data = convert_malignant_tweets_format(malignant_raw_data)

In [ ]:
len(malignant_data)

159571

In [ ]:
unbalanced_train_data['is_cyberbullying'].value_counts()

1    39747
0     7945
Name: is_cyberbullying, dtype: int64

In [ ]:
malignant_data['is_cyberbullying'].value_counts()

0    143346
1     16225
Name: is_cyberbullying, dtype: int64

**All bullying tweets can be added along with non bullying tweets**

In [ ]:
malignant_data['tweet_length'] = malignant_data['tweet_text'].str.len()
malignant_400_subset = malignant_data[malignant_data['tweet_length'] < 400]

In [ ]:
malignant_400_subset['is_cyberbullying'].value_counts()

0    102298
1     13488
Name: is_cyberbullying, dtype: int64

In [ ]:
malignant_bullying_400_subset = malignant_400_subset[malignant_400_subset['is_cyberbullying']==1]
malignant_not_bullying_400_subset = malignant_400_subset[malignant_400_subset['is_cyberbullying']==0]

In [ ]:
columns = ['tweet_text','is_cyberbullying']
new_train_data = pd.concat([malignant_bullying_400_subset[columns],unbalanced_train_data[columns]])
new_train_data['is_cyberbullying'].value_counts()

1    53235
0     7945
Name: is_cyberbullying, dtype: int64

**Add 45K non bullying tweets to balance the train data**

In [ ]:
additional_train_data_index = malignant_not_bullying_400_subset.sample(n=45000, random_state=23).index
transfer_malignant_data = malignant_not_bullying_400_subset.loc[additional_train_data_index]

In [ ]:
mixed_full_data = pd.concat([new_train_data[columns],transfer_malignant_data[columns]])

In [ ]:
mixed_full_data['is_cyberbullying'].value_counts()

1    53235
0    52945
Name: is_cyberbullying, dtype: int64

In [ ]:
new_train_data['is_cyberbullying'].value_counts()

1    53235
0     7945
Name: is_cyberbullying, dtype: int64

In [ ]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

In [ ]:
banned_list= string.punctuation
punctuation_reg_exp = "[" + banned_list + "]"
emoji_reg_exp = get_emoji_regexp()

def stemmer(text):
    tokenized = nltk.word_tokenize(text)
    ps = PorterStemmer()
    return ' '.join([ps.stem(words) for words in tokenized])

def clean_text(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)

    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    
    text = re.sub(punctuation_reg_exp,"",text)

    text = re.sub("\s\s+" , " ", text)

    text = re.sub(emoji_reg_exp, r"", text)

    text = stemmer(text)
    return text

In [ ]:
mixed_full_data['cleaned'] = mixed_full_data['tweet_text'].apply(clean_text)

In [ ]:
mixed_full_data['cleaned'].duplicated().sum()

3220

In [ ]:
cleaned_full_data = mixed_full_data.drop_duplicates("cleaned")
cleaned_full_data = cleaned_full_data[cleaned_full_data['cleaned']!='']

In [ ]:
cleaned_full_data['cleaned'].duplicated().sum()

0

**Create test dataset**

In [ ]:
def create_test_data(data, size=4000):
    not_malignant_tweets = data[data['is_cyberbullying'] == 0]
    malignant_tweets = data[data['is_cyberbullying'] == 1]
    not_malignant_subset = not_malignant_tweets.sample(n=size//2, random_state=23)
    malignant_subset = malignant_tweets.sample(n=size//2, random_state=23)
    test_data = pd.concat([not_malignant_subset,malignant_subset])
    return test_data

In [ ]:
mixed_test_data = create_test_data(cleaned_full_data, 10000)

In [ ]:
mixed_train_data = cleaned_full_data.drop(mixed_test_data.index,axis=0)

In [ ]:
mixed_test_data['is_cyberbullying'].value_counts()

0    5000
1    5000
Name: is_cyberbullying, dtype: int64

In [ ]:
mixed_train_data['is_cyberbullying'].value_counts()

0    46111
1    43868
Name: is_cyberbullying, dtype: int64

In [ ]:
if store_local:
    mixed_train_data.to_csv(local_path + "mixed_train_data.csv",index=False)
    mixed_test_data.to_csv(local_path + "mixed_test_data.csv",index=False)